## Данные и матрица интеракций

In [59]:
import pandas as pd
import numpy as np

data = pd.read_csv('recipes_normalized.csv')

data.head(10)

,url,name,ingredients,ingredients_normalized
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...","{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...","{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...","{'Фарш куриный': '800 г', 'Картофель': '1 кг',..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар-песок..."
5,https://www.povarenok.ru/recipes/show/167765/,Песочный пирог с тыквенным суфле,"{'Масло сливочное': '100 г', 'Сахар': '50 г', ...","{'Масло сливочное': '100 г', 'Сахар-песок': '5..."
6,https://www.povarenok.ru/recipes/show/100230/,Шоколадные конфеты ручной работы,"{'Какао-масло': '100 г', 'Какао тертое': '200 ...","{'Какао-масло': '100 г', 'Какао': '200 г', 'Си..."
7,https://www.povarenok.ru/recipes/show/96257/,Рыбно-тыквенный гратен,"{'Масло растительное': '3 ст. л.', 'Рыба': '40...","{'Масло растительное': '3 ст. л.', 'Рыба': '40..."
8,https://www.povarenok.ru/recipes/show/139360/,"Плов с креветками из риса ""Басмати""","{'Рис': '2 стак.', 'Вода': '4 стак.', 'Морковь...","{'Рис': '2 стак.', 'Вода': '4 стак.', 'Морковь..."
9,https://www.povarenok.ru/recipes/show/96774/,Молочное мороженое,"{'Молоко': '450 г', 'Желток яичный': '3 шт', '...","{'Молоко': '450 г', 'Желтки': '3 шт', 'Сахар-п..."


### Почистим данные 

In [60]:
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
import ast

all_ingredients = set()
for ingredients_str in data['ingredients_normalized']:
    ingredients_parsed = ast.literal_eval(ingredients_str)
    all_ingredients.update(ingredients_parsed.keys())

all_ingredients = sorted(list(all_ingredients))
ingredient_to_idx = {ing: idx for idx, ing in enumerate(all_ingredients)}

print(f"Ингредиентов: {len(all_ingredients)}")
print(f"Рецептов: {len(data)}")

Ингредиентов: 979
Рецептов: 146581


In [61]:
stop_words_drop = ['Соль', 'Сахар-песок', 'Перец черный молотый', 'Мука пшеничная',
'Сода', 'Сода гашеная уксусом']

In [62]:
from tqdm import tqdm

interactions = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    ingredients_parsed = ast.literal_eval(row['ingredients_normalized'])
    recipe_id = row.get('url', idx) 
        
    for ingredient in ingredients_parsed.keys():
        interactions.append((recipe_id, ingredient))

interactions_df = pd.DataFrame(interactions, columns=['recipe_id', 'ingredient_id'])
print(f"Interactions {len(interactions_df)}")

unique_recipes = interactions_df['recipe_id'].unique()
all_unique_ingredients = interactions_df['ingredient_id'].unique()
unique_ingredients = [
    ingredient for ingredient in all_unique_ingredients if ingredient not in stop_words_drop
]

recipe2id = {recipe: i for i, recipe in enumerate(unique_recipes)}
item2id = {ingredient: i for i, ingredient in enumerate(unique_ingredients)}

id2recipe = {i: recipe for recipe, i in recipe2id.items()}
id2item = {i: ingredient for ingredient, i in item2id.items()}

interactions_df['user_id'] = interactions_df['recipe_id'].map(recipe2id)
interactions_df['item_id'] = interactions_df['ingredient_id'].map(item2id)
interactions_df.dropna(subset=['item_id'], inplace=True)
interactions_df['item_id'] = interactions_df['item_id'].astype(int)


100%|██████████| 146581/146581 [00:21<00:00, 6927.78it/s] 


Interactions 1278324


In [63]:
print(len(unique_recipes), len(unique_ingredients))

146563 973


In [64]:
interactions_df.head(7)

,recipe_id,ingredient_id,user_id,item_id
0,https://www.povarenok.ru/recipes/show/164365/,Молоко,0,0
1,https://www.povarenok.ru/recipes/show/164365/,Клубника,0,1
3,https://www.povarenok.ru/recipes/show/1306/,Сыр твердый,1,2
4,https://www.povarenok.ru/recipes/show/1306/,Чеснок,1,3
5,https://www.povarenok.ru/recipes/show/1306/,Яйцо куриное,1,4
6,https://www.povarenok.ru/recipes/show/1306/,Грейпфрут,1,5
7,https://www.povarenok.ru/recipes/show/1306/,Лук зеленый,1,6


In [65]:
rows = interactions_df['user_id'].values
cols = interactions_df['item_id'].values
values = np.ones(len(rows), dtype=np.int8)

num_users= len(unique_recipes)
num_items= len(unique_ingredients)

interactions_matrix = csr_matrix(
    (values, (rows, cols)), 
    shape=(num_users, num_items)
)

In [66]:
print(f"Sparsity:{(1 -interactions_matrix.nnz /(num_users * num_items)) * 100:.2f}%")

Sparsity:99.26%


In [47]:
'''
import scipy.sparse as sp
import pickle

sp.save_npz('recsys_interactions.npz', interactions_matrix)

artifacts = {
    'recipe_to_cat': recipe_to_cat,
    'ingredient_to_cat': ingredient_to_cat,
    'cat_to_recipe': cat_to_recipe,
    'cat_to_ingredient': cat_to_ingredient,
    'unique_recipes': unique_recipes,
    'unique_ingredients': unique_ingredients
}

with open('model_artifacts.pkl', 'wb') as f:
    pickle.dump(artifacts, f)
print("model_artifacts.pkl")
'''

'\nimport scipy.sparse as sp\nimport pickle\n\nsp.save_npz(\'recsys_interactions.npz\', interactions_matrix)\n\nartifacts = {\n    \'recipe_to_cat\': recipe_to_cat,\n    \'ingredient_to_cat\': ingredient_to_cat,\n    \'cat_to_recipe\': cat_to_recipe,\n    \'cat_to_ingredient\': cat_to_ingredient,\n    \'unique_recipes\': unique_recipes,\n    \'unique_ingredients\': unique_ingredients\n}\n\nwith open(\'model_artifacts.pkl\', \'wb\') as f:\n    pickle.dump(artifacts, f)\nprint("model_artifacts.pkl")\n'

In [67]:
interactions_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(146563, 973), dtype=int8)

### Train test split

In [68]:
def train_val_test_split(
    interactions_df: pd.DataFrame,
    user_col: str = 'user_id',
    item_col: str = 'item_id',
    k_core: int = 3,
    test_size: float = 0.2,
    val_size: float = 0.1,
    seed: int = 42
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    while True:
        #взаимодействия для пользователей
        user_counts = interactions_df.groupby(user_col)[item_col].count()
        valid_users = user_counts[user_counts >= k_core].index
        
        #взаимодействия для айтемов
        item_counts = interactions_df.groupby(item_col)[user_col].count()
        valid_items = item_counts[item_counts >= k_core].index
        
        before_count = len(interactions_df)
        filtered_df = interactions_df[
            (interactions_df[user_col].isin(valid_users)) &
            (interactions_df[item_col].isin(valid_items))
        ]
        after_count = len(filtered_df)
        
        if before_count == after_count:
            break
        interactions_df = filtered_df

    print(f"Interactions left: {len(filtered_df)}")

    shuffled_df = filtered_df.sample(frac=1, random_state=seed)
    
    shuffled_df['user_interaction_rank'] = shuffled_df.groupby(user_col).cumcount()
    shuffled_df['user_interaction_count'] = shuffled_df.groupby(user_col)[user_col].transform('count')
    
    shuffled_df['is_test'] = shuffled_df['user_interaction_rank'] < (shuffled_df['user_interaction_count'] * test_size)
    
    train_val_df = shuffled_df[~shuffled_df['is_test']].copy()
    test_df = shuffled_df[shuffled_df['is_test']].copy()
    
    print(f"Train+val: {len(train_val_df)}")
    print(f"Test:{len(test_df)}")
    
    val_frac = val_size / (1 - test_size) 
    train_val_df['user_interaction_rank'] = train_val_df.groupby(user_col).cumcount()
    train_val_df['user_interaction_count'] = train_val_df.groupby(user_col)[user_col].transform('count')
    
    train_val_df['is_val'] = train_val_df['user_interaction_rank'] < (train_val_df['user_interaction_count'] * val_frac)
    
    train_df = train_val_df[~train_val_df['is_val']].copy()
    val_df = train_val_df[train_val_df['is_val']].copy()
    
    print(f"Train: {len(train_df)}")
    print(f"Val: {len(val_df)}")
    
    columns_to_drop = ['user_interaction_rank', 'user_interaction_count', 'is_test', 'is_val']
    train_df.drop(columns=columns_to_drop, inplace=True)
    val_df.drop(columns=columns_to_drop, inplace=True)
    test_df.drop(columns=columns_to_drop, errors ='ignore', inplace=True)

    print(f"total interactions: {len(train_df) + len(val_df) + len(test_df)}")
    print(f"unique users(recipes): train {train_df[user_col].nunique()}, val {val_df[user_col].nunique()}, test {test_df[user_col].nunique()} ")
    print(f"unique items: train {train_df[item_col].nunique()}, val {val_df[item_col].nunique()}, test {test_df[item_col].nunique()} ")

    return train_df, val_df, test_df

In [69]:
train, val, test = train_val_test_split(
    interactions_df, 
    k_core=5, 
    test_size=0.2,
    val_size=0.15   
)

print(train.shape)
print(val.shape)
print(test.shape)

Interactions left: 978048
Train+val: 731149
Test:246899
Train: 543085
Val: 188064
total interactions: 978048
unique users(recipes): train 123308, val 123308, test 123308 
unique items: train 848, val 824, test 831 
(543085, 4)
(188064, 4)
(246899, 4)


In [70]:
train.head()

,recipe_id,ingredient_id,user_id,item_id
300534,https://www.povarenok.ru/recipes/show/35463/,Масло сливочное,34416,15
1195993,https://www.povarenok.ru/recipes/show/97185/,Черника,137077,154
669888,https://www.povarenok.ru/recipes/show/33509/,Кефир,76716,93
855584,https://www.povarenok.ru/recipes/show/69306/,Молоко,98047,0
771076,https://www.povarenok.ru/recipes/show/64763/,Масло сливочное,88351,15


#### for eval

In [71]:
train_grouped = train.groupby('user_id')['item_id'].apply(list).reset_index()
train_grouped.rename(columns={'item_id': 'train_interactions'}, inplace=True)

val_grouped = val.groupby('user_id')['item_id'].apply(list).reset_index()
val_grouped.rename(columns={'item_id': 'val_interactions'}, inplace=True)

test_grouped = test.groupby('user_id')['item_id'].apply(list).reset_index()
test_grouped.rename(columns={'item_id': 'test_interactions'}, inplace=True)

train_val_joined = pd.merge(train_grouped, val_grouped, on='user_id', how='outer')
full_grouped_data = pd.merge(train_val_joined, test_grouped, on='user_id', how='outer')

In [72]:
from math import log2

def hit_rate(recommendations, ground_truth, k=100):
    """
    Calculate HitRate@k

    Args:
        recommendations: dict {user_id: list of recommended item_ids}
        ground_truth: dict {user_id: set of relevant item_ids}
        k: cutoff level
    """
    hits = 0
    total_users = len(recommendations)

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        if any(item in user_truth for item in user_recs):
            hits += 1

    return hits / total_users if total_users > 0 else 0.0


def precision(recommendations, ground_truth, k=100):
    """
    Calculate Precision@k
    """
    precisions = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_precision = relevant_count / k
        precisions.append(user_precision)

    return np.mean(precisions) if precisions else 0.0


def recall(recommendations, ground_truth, k=100):
    """
    Calculate Recall@k
    """
    recalls = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        if not user_truth:  # If no ground truth items, recall is 0
            recalls.append(0.0)
            continue

        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_recall = relevant_count / len(user_truth)
        recalls.append(user_recall)

    return np.mean(recalls) if recalls else 0.0

def mrr(recommendations, ground_truth, k=100):
    """
    Calculate MRR@k
    """
    reciprocal_ranks = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        user_rr = 0.0
        for rank, item in enumerate(user_recs, 1):
            if item in user_truth:
                user_rr = 1.0 / rank
                break

        reciprocal_ranks.append(user_rr)

    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0

def ndcg(recommendations, ground_truth, k=100, binary_relevance=True):
    """
    Calculate NDCG@k

    Args:
        binary_relevance: if True, uses binary relevance (0/1),
                         if False, expects relevance scores in ground_truth
    """
    ndcg_scores = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, {})

        # Calculate DCG
        dcg = 0.0
        for rank, item in enumerate(user_recs, 1):
            if binary_relevance:
                rel = 1.0 if item in user_truth else 0.0
            else:
                rel = user_truth.get(item, 0.0)

            dcg += rel / (log2(rank + 1) if rank == 1 else 1)

        # Calculate IDCG
        if binary_relevance:
            # For binary relevance, ideal is all 1's sorted first
            num_relevant = len(user_truth)
            ideal_gains = [1.0] * min(k, num_relevant)
        else:
            # For graded relevance, take top-k relevance scores
            ideal_gains = sorted(user_truth.values(), reverse=True)[:k]

        idcg = 0.0
        for rank, rel in enumerate(ideal_gains, 1):
            idcg += rel / (log2(rank + 1) if rank == 1 else 1)

        user_ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcg_scores.append(user_ndcg)

    return np.mean(ndcg_scores) if ndcg_scores else 0.0
        

In [73]:
def evaluate_model(df: pd.DataFrame, preds_col: str, gt_col: str, top_k: int = 20
) -> dict:
    recommendations = pd.Series(df[preds_col].values,index=df['user_id']).to_dict()

    ground_truth = pd.Series(df[gt_col].apply(set).values, index=df['user_id']).to_dict()

    hr = hit_rate(recommendations, ground_truth, k=top_k)
    p = precision(recommendations, ground_truth, k=top_k)
    r = recall(recommendations, ground_truth, k=top_k)
    m = mrr(recommendations, ground_truth, k=top_k)
    n = ndcg(recommendations, ground_truth, k=top_k)
    
    results = {
        f'hit_rate@{top_k}': hr,
        f'precision@{top_k}': p,
        f'recall@{top_k}': r,
        f'mrr@{top_k}': m,
        f'ndcg@{top_k}': n
    }
    
    return results

### model 1: EASE

In [74]:
from scipy import sparse as sps

matrix_train = sps.coo_matrix(
    (np.ones(train.shape[0]), (train['user_id'], train['item_id'])),
    shape=(len(recipe2id), len(item2id)),
)
matrix_train

<COOrdinate sparse matrix of dtype 'float64'
	with 543085 stored elements and shape (146563, 973)>

In [75]:
%%time

# Обучаем конечную модель
# Мы взяли реализацию из RecBole

def fit_ease(X, reg_weight=100):
    
    # gram matrix
    G = X.T @ X

    # add reg to diagonal
    G += reg_weight * sps.identity(G.shape[0])

    # convert to dense because inverse will be dense
    G = G.todense()

    # invert. this takes most of the time
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    # zero out diag
    np.fill_diagonal(B, 0.)
    
    return B

w = fit_ease(matrix_train)

CPU times: user 193 ms, sys: 93.7 ms, total: 287 ms
Wall time: 648 ms


In [76]:
def get_preds(user_interactions, item2id, id2item, model_weights):
    encoded_ids = user_interactions
    
    vector = np.zeros(len(item2id))
    vector[encoded_ids] = 1
    
    preds = vector @ model_weights
    preds[encoded_ids] = -np.inf  # Filter out items already seen
    
    top_indices = np.argsort(-preds)[:20]
    
    decoded = [id2item[i] for i in top_indices]
    
    return  top_indices

In [77]:
w = np.asarray(w)

tqdm.pandas()
full_grouped_data['ease_preds'] = full_grouped_data['train_interactions'].progress_apply(
    lambda interactions: get_preds(interactions, item2id, id2item, w)
)
full_grouped_data.head()

  0%|          | 0/123308 [00:00<?, ?it/s]

100%|██████████| 123308/123308 [00:48<00:00, 2538.72it/s]


,user_id,train_interactions,val_interactions,test_interactions,ease_preds
0,1,"[6, 3, 5, 7]",[4],"[2, 8]","[18, 49, 55, 11, 145, 8, 64, 77, 68, 4, 46, 12..."
1,2,"[11, 8, 10]",[3],[9],"[3, 39, 2, 4, 18, 46, 13, 16, 55, 48, 9, 70, 1..."
2,3,"[0, 2, 16, 18, 13, 10]","[12, 17]","[14, 15]","[4, 39, 3, 15, 8, 11, 48, 58, 151, 24, 181, 70..."
3,4,"[19, 20, 22]",[23],[21],"[4, 24, 67, 66, 159, 15, 23, 65, 29, 0, 18, 73..."
4,5,"[24, 25, 4, 29, 15]","[28, 22]","[26, 27]","[18, 65, 0, 151, 10, 35, 58, 22, 66, 61, 31, 1..."


In [78]:
evaluate_model(full_grouped_data, 'ease_preds', 'test_interactions', top_k=6)

{'hit_rate@6': 0.6114931715703766,
 'precision@6': np.float64(0.12720856175863152),
 'recall@6': np.float64(0.38461927314799793),
 'mrr@6': np.float64(0.37971677966284967),
 'ndcg@6': np.float64(0.38461927314799793)}

In [79]:
evaluate_model(full_grouped_data, 'ease_preds', 'test_interactions', top_k=10)

{'hit_rate@10': 0.7129869919226652,
 'precision@10': np.float64(0.09570344178804296),
 'recall@10': np.float64(0.48109854997242674),
 'mrr@10': np.float64(0.39212487854648137),
 'ndcg@10': np.float64(0.48109854997242674)}

In [80]:
evaluate_model(full_grouped_data, 'ease_preds', 'test_interactions', top_k=20)

{'hit_rate@20': 0.8289973075550654,
 'precision@20': np.float64(0.06166469328835113),
 'recall@20': np.float64(0.6180741179269256),
 'mrr@20': np.float64(0.4002903852493319),
 'ndcg@20': np.float64(0.6180741179269256)}

In [81]:
#навайбкодила как могла 
def inspect_recommendations(
    user_id: int, 
    df: pd.DataFrame, 
    id2recipe: dict, 
    id2item: dict,
    preds: str
):

    user_data = df[df['user_id'] == user_id] 
    user_data = user_data.iloc[0]
    recipe_name = id2recipe.get(user_id, "Unknown Recipe")
    train_items = [id2item.get(i, "Unknown") for i in user_data['train_interactions']]
    test_items = [id2item.get(i, "Unknown") for i in user_data['test_interactions']]
    pred_items = [id2item.get(i, "Unknown") for i in user_data[preds]]
    
    
    print("="*80)
    print(f"RECIPE: {recipe_name} (User ID: {user_id})")
    print("="*80)
    
    print(f"\n--- Ingredients in Training Set ({len(train_items)}) ---")
    print(", ".join(train_items))
    
    print(f"\n--- Ground Truth Ingredients in Test Set ({len(test_items)}) ---")
    print(", ".join(test_items))
    
    print(f"\n--- Top Recommended Ingredients ({len(pred_items)}) ---")
    successful_preds = set(test_items).intersection(set(pred_items))
    
    display_preds = []
    for item in pred_items:
        if item in successful_preds:
            display_preds.append(f"✅ {item}") # Add a checkmark for hits
        else:
            display_preds.append(item)
            
    print(", ".join(display_preds))
    print("\n" + "="*80)

inspect_recommendations(user_id=1, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')
inspect_recommendations(user_id=146561, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/1306/ (User ID: 1)

--- Ingredients in Training Set (4) ---
Лук зеленый, Чеснок, Грейпфрут, Салат

--- Ground Truth Ingredients in Test Set (2) ---
Сыр твердый, Майонез

--- Top Recommended Ingredients (20) ---
Масло растительное, Соевый соус, Огурец, Помидор, Укроп, ✅ Майонез, Горчица, Уксус, Сок лимона, Яйцо куриное, Перец болгарский, Петрушка, ✅ Сыр твердый, Лимон, Имбирь, Перец чили, Мед, Зелень, Томаты черри, Лук красный

RECIPE: https://www.povarenok.ru/recipes/show/167099/ (User ID: 146561)

--- Ingredients in Training Set (3) ---
Масло сливочное, Сыр сливочный, Ванильная эссенция

--- Ground Truth Ingredients in Test Set (1) ---
Разрыхлитель теста

--- Top Recommended Ingredients (20) ---
Яйцо куриное, Молоко, Сахарная пудра, ✅ Разрыхлитель теста, Сливки, Сахар коричневый, Желтки, Вода, Печенье, Корица, Сок лимона, Крахмал кукурузный, Шоколад темный, Сметана, Шоколад белый, Желатин, Дрожжи, Цедра лимона, Грецкий орех, Маскарпоне



In [82]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/169111/ (User ID: 29552)

--- Ingredients in Training Set (4) ---
Вода, Свекла, Дрожжи, Сок морковный

--- Ground Truth Ingredients in Test Set (2) ---
Яйцо куриное, Кунжут

--- Top Recommended Ingredients (20) ---
Масло растительное, Молоко, ✅ Яйцо куриное, Масло сливочное, Морковь, Картофель, Лук репчатый, Чеснок, Уксус, Майонез, Капуста белокочанная, Сельдь, Желтки, Томатная паста, ✅ Кунжут, Зелень, Мука ржаная, Яблоко, Сметана, Изюм



In [83]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/92453/ (User ID: 142283)

--- Ingredients in Training Set (7) ---
Баклажан, Морковь, Зелень, Картофель, Масло сливочное, Капуста белокочанная, Яйцо куриное

--- Ground Truth Ingredients in Test Set (3) ---
Томатная паста, Масло растительное, Лук репчатый

--- Top Recommended Ingredients (20) ---
✅ Лук репчатый, ✅ Масло растительное, Чеснок, Помидор, Перец болгарский, Сметана, Молоко, Майонез, Вода, Специи, Сыр твердый, Разрыхлитель теста, Кабачок, ✅ Томатная паста, Лист лавровый, Свекла, Уксус, Фарш, Грибы, Сливки



In [84]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/85178/ (User ID: 146102)

--- Ingredients in Training Set (3) ---
Корица, Хлопья овсяные, Кунжут

--- Ground Truth Ingredients in Test Set (2) ---
Цедра лимона, Банан

--- Top Recommended Ingredients (20) ---
Яйцо куриное, Масло растительное, Мед, Яблоко, Разрыхлитель теста, Масло сливочное, Вода, Сахар коричневый, Молоко, Дрожжи, Соевый соус, Имбирь, Грецкий орех, Орех мускатный, Гвоздика, Изюм, Орехи пекан, ✅ Банан, Семечки подсолнуха, Морковь



In [90]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/8150/ (User ID: 64550)

--- Ingredients in Training Set (3) ---
Маргарин, Разрыхлитель теста, Яблоко

--- Ground Truth Ingredients in Test Set (2) ---
Творог, Сметана

--- Top Recommended Ingredients (20) ---
Яйцо куриное, ✅ Сметана, Корица, Ванилин, Ванильный сахар, Молоко, Масло растительное, Сахарная пудра, Масло сливочное, Изюм, ✅ Творог, Сок лимона, Кефир, Грецкий орех, Какао, Майонез, Лимон, Желтки, Вода, Мед



### hyperparameters tuning

In [91]:
import optuna

def objective(trial):
    regul = trial.suggest_int('regul', 50, 2000)
    w = np.asarray(fit_ease(matrix_train, regul))
    tqdm.pandas()
    full_grouped_data['ease_preds'] = full_grouped_data['train_interactions'].progress_apply(
        lambda interactions: get_preds(interactions, item2id, id2item, w)
    )
    return evaluate_model(full_grouped_data, 'ease_preds', 'val_interactions', top_k=6)['hit_rate@6']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

[I 2025-11-14 15:34:52,888] A new study created in memory with name: no-name-8b309fc5-7c22-4115-aca6-27405a27dbe3
100%|██████████| 123308/123308 [00:38<00:00, 3173.16it/s]
[I 2025-11-14 15:35:37,158] Trial 0 finished with value: 0.5044684854186265 and parameters: {'regul': 326}. Best is trial 0 with value: 0.5044684854186265.
100%|██████████| 123308/123308 [00:18<00:00, 6646.10it/s]
[I 2025-11-14 15:35:57,999] Trial 1 finished with value: 0.5056525124079541 and parameters: {'regul': 208}. Best is trial 1 with value: 0.5056525124079541.
100%|██████████| 123308/123308 [00:18<00:00, 6548.39it/s]
[I 2025-11-14 15:36:22,547] Trial 2 finished with value: 0.4925065689168586 and parameters: {'regul': 1619}. Best is trial 1 with value: 0.5056525124079541.
100%|██████████| 123308/123308 [00:20<00:00, 6132.99it/s]
[I 2025-11-14 15:36:46,598] Trial 3 finished with value: 0.5075258701787394 and parameters: {'regul': 56}. Best is trial 3 with value: 0.5075258701787394.
100%|██████████| 123308/123308

In [92]:
study.best_trial.params['regul']

78

results on train + val (optimal parameters)

In [93]:
train_val_df = pd.concat([train, val])

num_users = len(recipe2id)
num_items = len(item2id)

interactions_matrix_train_val = sps.coo_matrix(
    (np.ones(train_val_df.shape[0]), (train_val_df['user_id'], train_val_df['item_id'])),
    shape=(num_users, num_items),
).tocsr()

In [94]:
w_final =fit_ease(interactions_matrix_train_val, reg_weight=study.best_trial.params['regul'])
w_final_array= np.asarray(w_final) 

train_val_interactions = full_grouped_data['train_interactions']

tqdm.pandas()
full_grouped_data['ease_full_preds'] = train_val_interactions.progress_apply(
    lambda interactions: get_preds(interactions, item2id, id2item, w_final_array)
)

100%|██████████| 123308/123308 [00:42<00:00, 2897.50it/s]


In [95]:
evaluate_model(df=full_grouped_data,preds_col='ease_full_preds',gt_col='test_interactions',top_k=6)

{'hit_rate@6': 0.6150858014078567,
 'precision@6': np.float64(0.12804251684129714),
 'recall@6': np.float64(0.38728130643051),
 'mrr@6': np.float64(0.3822151306754901),
 'ndcg@6': np.float64(0.38728130643051)}

In [96]:
evaluate_model(full_grouped_data, 'ease_preds', 'test_interactions', top_k=6)

{'hit_rate@6': 0.6111687806143965,
 'precision@6': np.float64(0.12715179334133497),
 'recall@6': np.float64(0.3844255847146982),
 'mrr@6': np.float64(0.3794937608806133),
 'ndcg@6': np.float64(0.3844255847146982)}

## slim

In [97]:
import warnings
warnings.filterwarnings('ignore')


In [98]:
from sklearn.linear_model import ElasticNet

def train_slim(
    train_matrix: sps.csr_matrix, 
    l1_reg: float = 0.001, 
    l2_reg: float = 0.0001
) -> sps.csr_matrix:
    num_items = train_matrix.shape[1]
    
    train_matrix_csc = train_matrix.tocsc()
    rows, cols, data = [], [], []

    model = ElasticNet(
        alpha=l1_reg + l2_reg,
        l1_ratio=l1_reg / (l1_reg + l2_reg) if (l1_reg + l2_reg) > 0 else 0,
        positive=True,
        fit_intercept=False,
        copy_X=False,   
        precompute=True, 
        max_iter=300, 
        tol=1e-4 
    )

    for j in tqdm(range(num_items)):
        y = train_matrix_csc[:, j].toarray().ravel()
        
        # (w_jj = 0) j-th column of the training data to zero
        start_pos = train_matrix_csc.indptr[j]
        end_pos = train_matrix_csc.indptr[j + 1]
        
        original_values = train_matrix_csc.data[start_pos:end_pos].copy()
        train_matrix_csc.data[start_pos:end_pos] = 0.0

        model.fit(train_matrix_csc, y)
        coeffs = model.coef_
        non_zero_indices = coeffs.nonzero()[0]
        if len(non_zero_indices) > 0:
            rows.extend(non_zero_indices)
            cols.extend([j] * len(non_zero_indices))
            data.extend(coeffs[non_zero_indices])
            
        train_matrix_csc.data[start_pos:end_pos] = original_values


    W_slim = sps.csr_matrix((data, (rows, cols)), shape=(num_items, num_items))
    
    return W_slim

In [99]:
w_slim = train_slim(matrix_train, l1_reg=0.001, l2_reg=0.001)

100%|██████████| 973/973 [02:04<00:00,  7.84it/s] 


In [100]:
def get_slim_predictions(user_interactions, model_weights, top_k=20):
    vector = np.zeros(model_weights.shape[1])
    vector[user_interactions] = 1
    
    scores = (sps.csr_matrix(vector) @ model_weights).toarray().flatten()
    
    scores[user_interactions] = -np.inf
    
    top_indices = np.argsort(-scores)[:top_k].tolist()
    
    return top_indices

In [101]:
tqdm.pandas()
full_grouped_data['slim_preds'] = full_grouped_data['train_interactions'].progress_apply(
    lambda interactions: get_slim_predictions(interactions, w_slim)
)
full_grouped_data.head()

100%|██████████| 123308/123308 [00:13<00:00, 8914.53it/s]


,user_id,train_interactions,val_interactions,test_interactions,ease_preds,ease_full_preds,slim_preds
0,1,"[6, 3, 5, 7]",[4],"[2, 8]","[18, 49, 55, 11, 145, 8, 64, 77, 68, 4, 46, 12...","[18, 49, 55, 11, 8, 145, 77, 64, 2, 159, 68, 1...","[18, 10, 145, 11, 4, 55, 49, 71, 8, 122, 39, 4..."
1,2,"[11, 8, 10]",[3],[9],"[3, 39, 2, 4, 18, 46, 13, 16, 55, 48, 9, 70, 1...","[2, 4, 39, 3, 46, 13, 18, 55, 16, 9, 70, 48, 1...","[18, 3, 4, 39, 2, 46, 13, 16, 55, 48, 122, 70,..."
2,3,"[0, 2, 16, 18, 13, 10]","[12, 17]","[14, 15]","[4, 39, 3, 15, 8, 11, 48, 58, 151, 24, 181, 70...","[4, 39, 3, 15, 11, 48, 151, 8, 181, 43, 58, 70...","[4, 39, 3, 15, 8, 11, 48, 24, 58, 151, 46, 70,..."
3,4,"[19, 20, 22]",[23],[21],"[4, 24, 67, 66, 15, 159, 23, 65, 29, 0, 18, 73...","[24, 4, 67, 159, 66, 23, 15, 18, 29, 27, 61, 7...","[67, 15, 4, 159, 65, 73, 23, 27, 150, 24, 18, ..."
4,5,"[24, 25, 4, 29, 15]","[28, 22]","[26, 27]","[18, 65, 0, 151, 10, 35, 58, 22, 66, 61, 31, 1...","[65, 18, 0, 151, 35, 10, 22, 58, 66, 61, 31, 1...","[18, 0, 65, 35, 58, 151, 66, 10, 31, 61, 142, ..."


In [102]:
evaluate_model(df=full_grouped_data,preds_col='slim_preds',gt_col='test_interactions',top_k=6)

{'hit_rate@6': 0.5592581178836734,
 'precision@6': np.float64(0.11321649852402114),
 'recall@6': np.float64(0.34191131151263504),
 'mrr@6': np.float64(0.34768871443864147),
 'ndcg@6': np.float64(0.34191131151263504)}

In [103]:
inspect_recommendations(user_id=1, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='slim_preds')

RECIPE: https://www.povarenok.ru/recipes/show/1306/ (User ID: 1)

--- Ingredients in Training Set (4) ---
Лук зеленый, Чеснок, Грейпфрут, Салат

--- Ground Truth Ingredients in Test Set (2) ---
Сыр твердый, Майонез

--- Top Recommended Ingredients (20) ---
Масло растительное, Лук репчатый, Укроп, Помидор, Яйцо куриное, Огурец, Соевый соус, Перец чили, ✅ Майонез, Петрушка, Морковь, Перец болгарский, Зелень, Картофель, Сок лимона, Уксус, Специи, Вода, ✅ Сыр твердый, Баклажан



In [104]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/100561/ (User ID: 19860)

--- Ingredients in Training Set (4) ---
Сахар тростниковый, Белок яичный, Творог, Тесто слоеное бездрожжевое

--- Ground Truth Ingredients in Test Set (2) ---
Желтки, Ванилин

--- Top Recommended Ingredients (20) ---
✅ Желтки, Яйцо куриное, Масло сливочное, Сахарная пудра, Ванильный сахар, ✅ Ванилин, Сметана, Яблоко, Разрыхлитель теста, Желатин, Молоко, Сливки, Сыр твердый, Сок лимона, Масло растительное, Вода, Шоколад темный, Крахмал кукурузный, Изюм, Корица



In [105]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/156571/ (User ID: 18674)

--- Ingredients in Training Set (3) ---
Масло сливочное, Сахарная пудра, Печенье

--- Ground Truth Ingredients in Test Set (2) ---
Какао, Молоко сгущенное

--- Top Recommended Ingredients (20) ---
Яйцо куриное, Сливки, Молоко, ✅ Какао, Творог, Сметана, Желатин, Ванильный сахар, Вода, ✅ Молоко сгущенное, Ванилин, Сыр сливочный, Шоколад темный, Грецкий орех, Шоколад молочный, Желтки, Сок лимона, Клубника, Белок яичный, Маскарпоне



In [106]:
random_user_id = full_grouped_data['user_id'].sample(1).iloc[0]
inspect_recommendations(user_id=random_user_id, df=full_grouped_data, id2recipe=id2recipe, id2item=id2item, preds='ease_preds')

RECIPE: https://www.povarenok.ru/recipes/show/72996/ (User ID: 86303)

--- Ingredients in Training Set (6) ---
Перец болгарский, Морковь, Рис, Масло растительное, Лук репчатый, Помидоры

--- Ground Truth Ingredients in Test Set (2) ---
Барбарис, Грибы

--- Top Recommended Ingredients (20) ---
Чеснок, Вода, Зелень, Помидор, Фарш, Перец чили, Яйцо куриное, Лист лавровый, Специи, Капуста белокочанная, Картофель, Петрушка, Томатная паста, Бульон, Базилик, Баклажан, Сельдерей черешковый, Паприка сладкая, Соевый соус, Филе куриное



In [107]:
import optuna

def objective(trial):
    l1_reg = trial.suggest_float('l1_reg', 1e-4, 1e-1)
    l2_reg = trial.suggest_float('l2_reg', 1e-4, 1e-1)
    w = train_slim(interactions_matrix, l1_reg=l1_reg, l2_reg=l2_reg)
    tqdm.pandas()
    full_grouped_data['slim_preds'] = full_grouped_data['train_interactions'].progress_apply(
    lambda interactions: get_slim_predictions(interactions, w)
    )
    return evaluate_model(full_grouped_data, 'slim_preds', 'val_interactions', top_k=6)['hit_rate@6']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

[I 2025-11-14 15:59:13,131] A new study created in memory with name: no-name-dc1af4dd-4e1d-4fa8-b06f-d843a7d3fcdf
100%|██████████| 123308/123308 [00:16<00:00, 7303.49it/s]
[I 2025-11-14 15:59:42,321] Trial 0 finished with value: 0.3837463911506147 and parameters: {'l1_reg': 0.017460008794477757, 'l2_reg': 0.09885147876475318}. Best is trial 0 with value: 0.3837463911506147.
100%|██████████| 123308/123308 [00:14<00:00, 8280.55it/s]
[I 2025-11-14 16:00:08,802] Trial 1 finished with value: 0.24804554449021962 and parameters: {'l1_reg': 0.06253394820420896, 'l2_reg': 0.0012256049209892351}. Best is trial 0 with value: 0.3837463911506147.
100%|██████████| 123308/123308 [00:29<00:00, 4142.06it/s]
[I 2025-11-14 16:00:58,828] Trial 2 finished with value: 0.3031920070068446 and parameters: {'l1_reg': 0.0440173743099334, 'l2_reg': 0.09212788917391393}. Best is trial 0 with value: 0.3837463911506147.
100%|██████████| 123308/123308 [00:37<00:00, 3301.25it/s]
[I 2025-11-14 16:01:48,143] Trial 3 fin

In [108]:
study.best_trial.params['l1_reg'], study.best_trial.params['l2_reg']

(0.0003948790736704186, 0.007761665029654702)

In [109]:
w_slim = train_slim(interactions_matrix_train_val, l1_reg=study.best_trial.params['l1_reg'],
l2_reg=study.best_trial.params['l2_reg'])

100%|██████████| 973/973 [03:17<00:00,  4.93it/s] 


In [110]:
tqdm.pandas()
full_grouped_data['slim_full_preds'] = full_grouped_data['train_interactions'].progress_apply(
    lambda interactions: get_slim_predictions(interactions, w_slim)
)
full_grouped_data.head()

100%|██████████| 123308/123308 [00:20<00:00, 5980.76it/s]


,user_id,train_interactions,val_interactions,test_interactions,ease_preds,ease_full_preds,slim_preds,slim_full_preds
0,1,"[6, 3, 5, 7]",[4],"[2, 8]","[18, 49, 55, 11, 145, 8, 64, 77, 68, 4, 46, 12...","[18, 49, 55, 11, 8, 145, 77, 64, 2, 159, 68, 1...","[18, 55, 145, 11, 49, 4, 122, 8, 71, 77, 68, 6...","[18, 55, 11, 49, 145, 8, 4, 122, 68, 71, 64, 1..."
1,2,"[11, 8, 10]",[3],[9],"[3, 39, 2, 4, 18, 46, 13, 16, 55, 48, 9, 70, 1...","[2, 4, 39, 3, 46, 13, 18, 55, 16, 9, 70, 48, 1...","[4, 18, 2, 39, 3, 46, 13, 55, 16, 70, 48, 43, ...","[18, 4, 2, 3, 39, 46, 13, 16, 55, 48, 70, 9, 1..."
2,3,"[0, 2, 16, 18, 13, 10]","[12, 17]","[14, 15]","[4, 39, 3, 15, 8, 11, 48, 58, 151, 24, 181, 70...","[4, 39, 3, 15, 11, 48, 151, 8, 181, 43, 58, 70...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 43, 70,...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 70, 43,..."
3,4,"[19, 20, 22]",[23],[21],"[4, 24, 67, 66, 15, 159, 23, 65, 29, 0, 18, 73...","[24, 4, 67, 159, 66, 23, 15, 18, 29, 27, 61, 7...","[67, 24, 15, 159, 65, 4, 23, 73, 27, 29, 150, ...","[67, 4, 15, 65, 159, 23, 24, 73, 27, 150, 29, ..."
4,5,"[24, 25, 4, 29, 15]","[28, 22]","[26, 27]","[18, 65, 0, 151, 10, 35, 58, 22, 66, 61, 31, 1...","[65, 18, 0, 151, 35, 10, 22, 58, 66, 61, 31, 1...","[18, 0, 65, 35, 151, 58, 22, 66, 61, 31, 142, ...","[18, 0, 65, 35, 58, 151, 22, 66, 31, 61, 10, 1..."


In [111]:
evaluate_model(df=full_grouped_data,preds_col='slim_full_preds',gt_col='test_interactions',top_k=6)

{'hit_rate@6': 0.5832711584001038,
 'precision@6': np.float64(0.11919340188795537),
 'recall@6': np.float64(0.36059325699332834),
 'mrr@6': np.float64(0.3603429353056304),
 'ndcg@6': np.float64(0.36059325699332834)}

### toppopular as baseline

In [112]:
class TopPopular:
    def __init__(self):
        self.recommendations = None
        self.trained = False

    def fit(self, df: pd.DataFrame, item_col: str = "item_id") -> None:
        self.recommendations = df[item_col].value_counts().index.to_numpy()
        self.trained = True

    def predict(self, user_ids: list, top_k: int = 20) -> list:
        if not self.trained:
            raise RuntimeError("You must fit the model before making predictions.")
        
        top_k_recs = self.recommendations[:top_k].tolist()
        return [top_k_recs] * len(user_ids)

toppop_model = TopPopular()
toppop_model.fit(train, item_col='item_id')

In [113]:
pop_predictions = toppop_model.predict( user_ids=full_grouped_data['user_id'], top_k=20)

full_grouped_data['toppop_preds'] = pop_predictions

In [114]:
evaluate_model(df=full_grouped_data,preds_col='toppop_preds',gt_col='test_interactions',top_k=6)

{'hit_rate@6': 0.42497648165569146,
 'precision@6': np.float64(0.07926628172273223),
 'recall@6': np.float64(0.2413259209999892),
 'mrr@6': np.float64(0.21603031974135228),
 'ndcg@6': np.float64(0.2413259209999892)}

In [115]:
full_grouped_data.head()

,user_id,train_interactions,val_interactions,test_interactions,ease_preds,ease_full_preds,slim_preds,slim_full_preds,toppop_preds
0,1,"[6, 3, 5, 7]",[4],"[2, 8]","[18, 49, 55, 11, 145, 8, 64, 77, 68, 4, 46, 12...","[18, 49, 55, 11, 8, 145, 77, 64, 2, 159, 68, 1...","[18, 55, 145, 11, 49, 4, 122, 8, 71, 77, 68, 6...","[18, 55, 11, 49, 145, 8, 4, 122, 68, 71, 64, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
1,2,"[11, 8, 10]",[3],[9],"[3, 39, 2, 4, 18, 46, 13, 16, 55, 48, 9, 70, 1...","[2, 4, 39, 3, 46, 13, 18, 55, 16, 9, 70, 48, 1...","[4, 18, 2, 39, 3, 46, 13, 55, 16, 70, 48, 43, ...","[18, 4, 2, 3, 39, 46, 13, 16, 55, 48, 70, 9, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
2,3,"[0, 2, 16, 18, 13, 10]","[12, 17]","[14, 15]","[4, 39, 3, 15, 8, 11, 48, 58, 151, 24, 181, 70...","[4, 39, 3, 15, 11, 48, 151, 8, 181, 43, 58, 70...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 43, 70,...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 70, 43,...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
3,4,"[19, 20, 22]",[23],[21],"[4, 24, 67, 66, 15, 159, 23, 65, 29, 0, 18, 73...","[24, 4, 67, 159, 66, 23, 15, 18, 29, 27, 61, 7...","[67, 24, 15, 159, 65, 4, 23, 73, 27, 29, 150, ...","[67, 4, 15, 65, 159, 23, 24, 73, 27, 150, 29, ...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
4,5,"[24, 25, 4, 29, 15]","[28, 22]","[26, 27]","[18, 65, 0, 151, 10, 35, 58, 22, 66, 61, 31, 1...","[65, 18, 0, 151, 35, 10, 22, 58, 66, 61, 31, 1...","[18, 0, 65, 35, 151, 58, 22, 66, 61, 31, 142, ...","[18, 0, 65, 35, 58, 151, 22, 66, 31, 61, 10, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."


In [116]:
top_20_ids = pop_predictions[0][:20]

print("TopPopular 20")
for item_id in top_20_ids:
    print(id2item.get(item_id))

TopPopular 20
Масло растительное
Яйцо куриное
Лук репчатый
Масло сливочное
Чеснок
Вода
Морковь
Молоко
Сметана
Майонез
Сыр твердый
Помидор
Картофель
Разрыхлитель теста
Зелень
Сливки
Перец болгарский
Соевый соус
Специи
Сок лимона


In [117]:
inspect_recommendations(user_id=145, df=full_grouped_data, 
id2recipe=id2recipe, id2item=id2item, preds='toppop_preds')

RECIPE: https://www.povarenok.ru/recipes/show/142765/ (User ID: 145)

--- Ingredients in Training Set (4) ---
Молоко, Какао, Ягода, Сливки

--- Ground Truth Ingredients in Test Set (2) ---
Банан, Крупа манная

--- Top Recommended Ingredients (20) ---
Масло растительное, Яйцо куриное, Лук репчатый, Масло сливочное, Чеснок, Вода, Морковь, Молоко, Сметана, Майонез, Сыр твердый, Помидор, Картофель, Разрыхлитель теста, Зелень, Сливки, Перец болгарский, Соевый соус, Специи, Сок лимона

